In [1]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [11]:
DATASET_PATH  = '.'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 2
BATCH_SIZE    = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 20
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [12]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/validation',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 15724 images belonging to 2 classes.
Found 1964 images belonging to 2 classes.


In [13]:
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(net_final.summary())

# train the model
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# save trained weights
net_final.save(WEIGHTS_FINAL)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_49 (Activation)      (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

  42/1965 [..............................] - ETA: 28:38 - loss: 0.7489 - acc: 0.6369

/home/ki/miniconda3/envs/cuda/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3473408 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/ki/miniconda3/envs/cuda/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 11. 
  warnings.warn(str(msg))


 288/1965 [===>..........................] - ETA: 8:27 - loss: 0.4760 - acc: 0.7817

/home/ki/miniconda3/envs/cuda/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2818048 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


...

/home/ki/miniconda3/envs/cuda/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


1536/1965 [======================>.......] - ETA: 1:34 - loss: 0.2751 - acc: 0.8843

/home/ki/miniconda3/envs/cuda/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1114112 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


1965/1965 [==============================] - 438s 223ms/step - loss: 0.2514 - acc: 0.8954 - val_loss: 0.0552 - val_acc: 0.9760
Epoch 2/20
1965/1965 [==============================] - 409s 208ms/step - loss: 0.1469 - acc: 0.9431 - val_loss: 0.0438 - val_acc: 0.9821
Epoch 3/20
1965/1965 [==============================] - 410s 209ms/step - loss: 0.0994 - acc: 0.9627 - val_loss: 0.0397 - val_acc: 0.9857
Epoch 4/20
 388/1965 [====>.........................] - ETA: 5:24 - loss: 0.0797 - acc: 0.9688

KeyboardInterrupt: 

In [16]:
model = net_final

In [18]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
        "test2",
        target_size=(224,224),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

Found 1962 images belonging to 1 classes.


In [30]:
fileNames = []
for fileName in test_generator.filenames:
    real_name = fileName.split("/")[1]
    fileNames.append(real_name)

In [32]:
import pandas as pd
standard = pd.read_csv("./sampleSubmission.csv")

In [39]:
import numpy as np
standard['Category'] = np.where(predict>0.5,0,1)

In [40]:
standard["Id"] = np.array(fileNames)

In [41]:
standard.to_csv("./sample.csv")